In [1]:
import os
import base64
from io import BytesIO
import requests
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [17]:

def identify_product_from_image(image_path_or_url):
    """
    Identifies a product from an image using LangChain with Gemini.

    Args:
        image_path_or_url: The local file path or URL of the product image.

    Returns:
        A string containing the identified product information or an error message.
    """
    try:
        # --- 1. Load Image ---
        if image_path_or_url.startswith(("http://", "https://")):
            # Handle URL
            response = requests.get(image_path_or_url)
            response.raise_for_status()
            image_bytes = response.content
        else:
            # Handle local file path
            with open(image_path_or_url, "rb") as image_file:
                image_bytes = image_file.read()

        # --- 2. Prepare the Prompt and Image for LangChain ---
        prompt_text = """
        Analyze the provided image of a product and return the following information in a structured format:
        1.  **Product Name:** The primary name of the product.
        2.  **Product Type:** The category of the product (e.g., Food, Cosmetic, Cleaning Supply).
        3.  **Extracted Information:** All visible text for ingredients, labels, and warnings and certifications.
        4.  **Brand:** The brand name of the product.
        5.  **Summary:** A brief one-sentence summary of the product.
        """
        
        image_b64 = base64.b64encode(image_bytes).decode("utf-8")

        message = HumanMessage(
            content=[
                {"type": "text", "text": prompt_text},
                {
                    "type": "image_url",
                    "image_url": f"data:image/jpeg;base64,{image_b64}",
                },
            ]
        )

        # --- 3. Call the Gemini API via LangChain ---
        llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
        response = llm.invoke([message])

        # --- 4. Process and Return the Response ---
        if response and response.content:
            return response.content
        else:
            return "Failed to get a valid response from the Gemini API."

    except FileNotFoundError:
        return f"Error: Image file not found at: {image_path_or_url}"
    except requests.exceptions.RequestException as e:
        return f"Error: Failed to fetch image from URL: {e}"
    except Exception as e:
        return f"An unexpected error occurred: {e}"

# --- Example Usage ---

if __name__ == "__main__":
    # --- Get Image from User ---
    print("Please provide the path to the product image.")
    image_location = input("Enter local file path or image URL: ").strip()

    if not image_location:
        print("No image location provided. Exiting.")
    else:
        # --- Identify Product ---
        print("\nAnalyzing product... Please wait.")
        product_info = identify_product_from_image(image_location)

        # --- Display Results ---
        print("\n--- Product Identification Results ---")
        print(product_info)
        print("\n------------------------------------")


Please provide the path to the product image.

Analyzing product... Please wait.

--- Product Identification Results ---
Here's the structured information extracted from the product image:

1.  **Product Name:** Kurkure
2.  **Product Type:** Snack (Namkeen), Food
3.  **Extracted Information:**
    *   **Brand Statement:** "Kurkure is a Registered Trade Mark of PepsiCo, Inc."
    *   **Manufacturer Details:** "MFD. BY: For manufacturer's details, see first two characters of batch no. and see below."
        *   N1 - PepsiCo India Holdings Pvt. Ltd., Lic. No. 10012063000110.
        *   N2 - PepsiCo India Holdings Pvt. Ltd., Lic. No. 10012022000339.
        *   N3 - PepsiCo India Holdings Pvt. Ltd., Lic. No. 10012031000120.
        *   DF - Daawat Foods Ltd., Lic. No. 10017071000922.
        *   HB - Heemankshi Bakers Pvt. Ltd., Lic. No. 10014047000100.
        *   AB - AFP Manufacturing Co. Pvt. Ltd., Lic. No. 10016033000284.
        *   JD - M/S JDB STEEL LLP, Lic. No. 10017071000922.
